In [3]:
# Cell 1: Import Libraries and Setup
import cv2
import numpy as np
import hashlib
import pickle
import os
import time
import json
import uuid
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cosine, euclidean
import warnings
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance
import base64
from datetime import datetime
from io import BytesIO
warnings.filterwarnings('ignore')

# Set up matplotlib for better visualization
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

print("All libraries imported successfully!")

# Cell 2: Utility Functions for Output Formatting
def print_section_header(title):
    """Print a formatted section header"""
    print("\n" + "="*80)
    print(f" {title} ")
    print("="*80)

def print_step(step_num, description):
    """Print a formatted step"""
    print(f"\n[STEP {step_num}] {description}")
    print("-" * 60)

def print_result(key, value):
    """Print a key-value result pair"""
    print(f"  ✓ {key}: {value}")

print("Utility functions defined!")

# Cell 3: Main Elephant Face ID Class - Part 1 (Initialization)
class ElephantFaceIDDetailed:
    def __init__(self, model_dir="elephant_models", dataset_path="D:\\Downloads\\Elephants"):
        """Initialize the Elephant Face ID system with detailed output"""
        print_section_header("INITIALIZING ELEPHANT FACE ID SYSTEM")
        
        self.model_dir = Path(model_dir)
        self.model_dir.mkdir(exist_ok=True)
        self.dataset_path = Path(dataset_path)
        
        # Create directories for thesis comparison images
        self.thesis_output_dir = Path("thesis_comparisons")
        self.thesis_output_dir.mkdir(exist_ok=True)
        
        self.alignment_comparison_dir = self.thesis_output_dir / "face_alignment_normalization"
        self.alignment_comparison_dir.mkdir(exist_ok=True)
        
        self.augmentation_comparison_dir = self.thesis_output_dir / "view_augmentation_pose_invariance"
        self.augmentation_comparison_dir.mkdir(exist_ok=True)
        
        print_result("Model Directory", self.model_dir)
        print_result("Dataset Path", self.dataset_path)
        print_result("Thesis Output Directory", self.thesis_output_dir)
        print_result("Alignment Comparison Directory", self.alignment_comparison_dir)
        print_result("Augmentation Comparison Directory", self.augmentation_comparison_dir)
        
        # File paths for models and data
        self.pca_model_path = self.model_dir / "pca_features.pkl"
        self.scaler_path = self.model_dir / "feature_scaler.pkl"
        self.db_path = self.model_dir / "elephants_database.pkl"
        
        print_result("PCA Model Path", self.pca_model_path)
        print_result("Scaler Path", self.scaler_path)
        print_result("Database Path", self.db_path)
        
        # Matching threshold
        self.match_threshold = 0.55
        print_result("Match Threshold", self.match_threshold)
        
        # Load existing models
        self.pca = None
        self.scaler = None
        self._load_models()
        
        # Load elephant database
        self.elephants_db = {}
        self._load_database()
        
        # Initialize collections
        self.collected_features = []
        self.all_processed_images = []
        self.ground_truth = {}
        
        # Counter for thesis image naming
        self.thesis_image_counter = 0
        
        print_result("Initialization Status", "COMPLETE")

print("ElephantFaceIDDetailed class initialization defined!")

# Cell 4: Model and Database Loading Methods
# Add these methods to the ElephantFaceIDDetailed class
def _load_models(self):
    """Load pre-trained models with detailed output"""
    print_step(1, "Loading Pre-trained Models")
    
    if self.pca_model_path.exists() and self.scaler_path.exists():
        try:
            with open(self.pca_model_path, 'rb') as f:
                self.pca = pickle.load(f)
            print_result("PCA Model", "LOADED SUCCESSFULLY")
            print_result("PCA Components", f"{self.pca.n_components_} components")
            print_result("PCA Explained Variance", f"{sum(self.pca.explained_variance_ratio_):.4f}")
            
            with open(self.scaler_path, 'rb') as f:
                self.scaler = pickle.load(f)
            print_result("Feature Scaler", "LOADED SUCCESSFULLY")
            print_result("Scaler Mean Shape", self.scaler.mean_.shape)
            print_result("Scaler Scale Shape", self.scaler.scale_.shape)
            
        except Exception as e:
            print_result("Model Loading Error", str(e))
            self.pca = None
            self.scaler = None
    else:
        print_result("Pre-trained Models", "NOT FOUND - Will create new ones")

def _load_database(self):
    """Load elephant database with detailed output"""
    print_step(2, "Loading Elephant Database")
    
    if self.db_path.exists():
        try:
            with open(self.db_path, 'rb') as f:
                self.elephants_db = pickle.load(f)
            print_result("Database Status", "LOADED SUCCESSFULLY")
            print_result("Unique Elephants", len(self.elephants_db))
            
            # Show database details
            total_embeddings = sum(len(embeddings) for embeddings in self.elephants_db.values())
            print_result("Total Embeddings", total_embeddings)
            
            if self.elephants_db:
                print("\n  Database Contents:")
                for i, (elephant_id, embeddings) in enumerate(self.elephants_db.items(), 1):
                    print(f"    {i}. {elephant_id}: {len(embeddings)} embeddings")
                    
        except Exception as e:
            print_result("Database Loading Error", str(e))
            self.elephants_db = {}
    else:
        print_result("Database Status", "NOT FOUND - Will create new one")

# Add methods to class
ElephantFaceIDDetailed._load_models = _load_models
ElephantFaceIDDetailed._load_database = _load_database

print("Model and database loading methods defined!")

# Cell 5: Image Preprocessing Methods
def preprocess_image(self, image):
    """Preprocess image with detailed output"""
    print_step(3, "Preprocessing Image")
    
    original_shape = image.shape
    print_result("Original Image Shape", original_shape)
    
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        print_result("Color Conversion", "BGR to Grayscale")
    else:
        gray = image.copy()
        print_result("Color Conversion", "Already Grayscale")
    
    print_result("Grayscale Shape", gray.shape)
    print_result("Pixel Value Range", f"{gray.min()} - {gray.max()}")
    
    # Apply CLAHE for better contrast
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    print_result("CLAHE Enhancement", "Applied (clipLimit=3.0, tileGridSize=(8,8))")
    print_result("Enhanced Pixel Range", f"{enhanced.min()} - {enhanced.max()}")
    
    # Apply denoising
    denoised = cv2.fastNlMeansDenoising(enhanced, None, 5, 7, 21)
    print_result("Denoising", "Applied (h=5, templateWindowSize=7, searchWindowSize=21)")
    print_result("Final Pixel Range", f"{denoised.min()} - {denoised.max()}")
    
    return denoised, gray

# Add to class
ElephantFaceIDDetailed.preprocess_image = preprocess_image

print("Image preprocessing methods defined!")

# Cell 6: Face Detection Methods
def detect_elephant_face(self, image):
    """Detect elephant face with detailed output"""
    print_step(4, "Detecting Elephant Face")
    
    preprocessed, gray = self.preprocess_image(image)
    h, w = preprocessed.shape
    print_result("Preprocessed Image Size", f"{w} x {h}")
    
    # Create directory for face detection outputs
    self.face_detection_output_dir = self.thesis_output_dir / "face_detection_steps"
    self.face_detection_output_dir.mkdir(exist_ok=True)
    
    face_candidates = []
    
    # Method 1: Thresholding and contour detection
    print("\n  Method 1: Contour-based Detection")
    try:
        _, binary = cv2.threshold(preprocessed, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        print_result("Otsu Threshold", "Applied")
        print_result("Binary Pixel Distribution", f"White: {np.sum(binary == 255)}, Black: {np.sum(binary == 0)}")
        
        # Save Method 1 binary image
        method1_binary_filename = f"{self.thesis_image_counter:03d}_method1_binary.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method1_binary_filename), binary)
        print_result("Method 1 Binary Saved", str(self.face_detection_output_dir / method1_binary_filename))
        
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print_result("Total Contours Found", len(contours))
        
        # Create visualization for Method 1
        method1_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(method1_viz, contours, -1, (0, 255, 0), 2)
        
        valid_contours = 0
        for contour in contours:
            x, y, w_cont, h_cont = cv2.boundingRect(contour)
            aspect_ratio = float(w_cont) / h_cont
            area_ratio = w_cont * h_cont / float(preprocessed.shape[0] * preprocessed.shape[1])
            
            if 0.5 <= aspect_ratio <= 2.0 and w_cont > 50 and h_cont > 50:
                if 0.1 <= area_ratio <= 0.9:
                    face_candidates.append((x, y, x+w_cont, y+h_cont))
                    valid_contours += 1
                    print_result(f"Valid Contour {valid_contours}", f"({x}, {y}, {x+w_cont}, {y+h_cont}) - AR: {aspect_ratio:.2f}, Area: {area_ratio:.3f}")
                    # Draw valid rectangles
                    cv2.rectangle(method1_viz, (x, y), (x+w_cont, y+h_cont), (255, 0, 0), 3)
        
        # Save Method 1 visualization
        method1_viz_filename = f"{self.thesis_image_counter:03d}_method1_contours.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method1_viz_filename), method1_viz)
        print_result("Method 1 Visualization Saved", str(self.face_detection_output_dir / method1_viz_filename))
        
        print_result("Method 1 Candidates", len(face_candidates))
        
    except Exception as e:
        print_result("Method 1 Error", str(e))
    
    # If no candidates from method 1, try method 2
    if not face_candidates:
        print("\n  Method 2: Edge-based Detection")
        try:
            edges = cv2.Canny(preprocessed, 50, 150)
            edge_pixels = np.sum(edges > 0)
            print_result("Edge Pixels Found", edge_pixels)
            
            # Save Method 2 edge image
            method2_edges_filename = f"{self.thesis_image_counter:03d}_method2_edges.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_edges_filename), edges)
            print_result("Method 2 Edges Saved", str(self.face_detection_output_dir / method2_edges_filename))
            
            kernel = np.ones((3, 3), np.uint8)
            dilated = cv2.dilate(edges, kernel, iterations=1)
            print_result("Edge Dilation", "Applied")
            
            # Save Method 2 dilated edges
            method2_dilated_filename = f"{self.thesis_image_counter:03d}_method2_dilated.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_dilated_filename), dilated)
            print_result("Method 2 Dilated Saved", str(self.face_detection_output_dir / method2_dilated_filename))
            
            contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            print_result("Edge Contours Found", len(contours))
            
            # Create visualization for Method 2
            method2_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
            
            valid_edge_contours = 0
            for contour in contours:
                x, y, w_cont, h_cont = cv2.boundingRect(contour)
                area = w_cont * h_cont
                img_area = preprocessed.shape[0] * preprocessed.shape[1]
                
                if 0.05 * img_area < area < 0.9 * img_area:
                    face_candidates.append((x, y, x+w_cont, y+h_cont))
                    valid_edge_contours += 1
                    print_result(f"Edge Contour {valid_edge_contours}", f"({x}, {y}, {x+w_cont}, {y+h_cont}) - Area: {area}")
                    # Draw valid rectangles
                    cv2.rectangle(method2_viz, (x, y), (x+w_cont, y+h_cont), (0, 255, 255), 3)
            
            # Sort by area (largest first)
            if face_candidates:
                face_candidates.sort(key=lambda rect: (rect[2]-rect[0])*(rect[3]-rect[1]), reverse=True)
                face_candidates = face_candidates[:3]  # Keep top 3
            
            # Save Method 2 visualization
            method2_viz_filename = f"{self.thesis_image_counter:03d}_method2_detection.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_viz_filename), method2_viz)
            print_result("Method 2 Visualization Saved", str(self.face_detection_output_dir / method2_viz_filename))
            
            print_result("Method 2 Candidates", len(face_candidates))
            
        except Exception as e:
            print_result("Method 2 Error", str(e))
    
    # Fallback method if no faces found
    if not face_candidates:
        print("\n  Method 3: Center Fallback Detection")
        center_x, center_y = w // 2, h // 2
        face_size = min(w, h) // 2
        fallback_rect = (center_x - face_size, center_y - face_size, 
                       center_x + face_size, center_y + face_size)
        face_candidates = [fallback_rect]
        print_result("Fallback Rectangle", f"({fallback_rect[0]}, {fallback_rect[1]}, {fallback_rect[2]}, {fallback_rect[3]})")
        
        # Create visualization for Method 3
        method3_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
        cv2.rectangle(method3_viz, (fallback_rect[0], fallback_rect[1]), 
                     (fallback_rect[2], fallback_rect[3]), (255, 0, 255), 3)
        cv2.putText(method3_viz, "FALLBACK CENTER", (fallback_rect[0], fallback_rect[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)
        
        # Save Method 3 visualization
        method3_viz_filename = f"{self.thesis_image_counter:03d}_method3_fallback.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method3_viz_filename), method3_viz)
        print_result("Method 3 Visualization Saved", str(self.face_detection_output_dir / method3_viz_filename))
    
    # Save final detection result with all candidates
    final_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
    for i, (x1, y1, x2, y2) in enumerate(face_candidates):
        color = (0, 255, 0) if i == 0 else (0, 165, 255)  # Green for primary, orange for others
        thickness = 3 if i == 0 else 2
        cv2.rectangle(final_viz, (x1, y1), (x2, y2), color, thickness)
        cv2.putText(final_viz, f"Face {i+1}", (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    
    final_detection_filename = f"{self.thesis_image_counter:03d}_final_detection.jpg"
    cv2.imwrite(str(self.face_detection_output_dir / final_detection_filename), final_viz)
    print_result("Final Detection Saved", str(self.face_detection_output_dir / final_detection_filename))
    
    print_result("Final Candidates Found", len(face_candidates))
    
    return face_candidates, preprocessed

# Update the method in the class
ElephantFaceIDDetailed.detect_elephant_face = detect_elephant_face
def detect_elephant_face(self, image):
    """Detect elephant face with detailed output"""
    print_step(4, "Detecting Elephant Face")
    
    preprocessed, gray = self.preprocess_image(image)
    h, w = preprocessed.shape
    print_result("Preprocessed Image Size", f"{w} x {h}")
    
    # Create directory for face detection outputs
    self.face_detection_output_dir = self.thesis_output_dir / "face_detection_steps"
    self.face_detection_output_dir.mkdir(exist_ok=True)
    
    face_candidates = []
    
    # Method 1: Thresholding and contour detection
    print("\n  Method 1: Contour-based Detection")
    try:
        _, binary = cv2.threshold(preprocessed, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        print_result("Otsu Threshold", "Applied")
        print_result("Binary Pixel Distribution", f"White: {np.sum(binary == 255)}, Black: {np.sum(binary == 0)}")
        
        # Save Method 1 binary image
        method1_binary_filename = f"{self.thesis_image_counter:03d}_method1_binary.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method1_binary_filename), binary)
        print_result("Method 1 Binary Saved", str(self.face_detection_output_dir / method1_binary_filename))
        
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print_result("Total Contours Found", len(contours))
        
        # Create visualization for Method 1
        method1_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(method1_viz, contours, -1, (0, 255, 0), 2)
        
        valid_contours = 0
        for contour in contours:
            x, y, w_cont, h_cont = cv2.boundingRect(contour)
            aspect_ratio = float(w_cont) / h_cont
            area_ratio = w_cont * h_cont / float(preprocessed.shape[0] * preprocessed.shape[1])
            
            if 0.5 <= aspect_ratio <= 2.0 and w_cont > 50 and h_cont > 50:
                if 0.1 <= area_ratio <= 0.9:
                    face_candidates.append((x, y, x+w_cont, y+h_cont))
                    valid_contours += 1
                    print_result(f"Valid Contour {valid_contours}", f"({x}, {y}, {x+w_cont}, {y+h_cont}) - AR: {aspect_ratio:.2f}, Area: {area_ratio:.3f}")
                    # Draw valid rectangles
                    cv2.rectangle(method1_viz, (x, y), (x+w_cont, y+h_cont), (255, 0, 0), 3)
        
        # Save Method 1 visualization
        method1_viz_filename = f"{self.thesis_image_counter:03d}_method1_contours.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method1_viz_filename), method1_viz)
        print_result("Method 1 Visualization Saved", str(self.face_detection_output_dir / method1_viz_filename))
        
        print_result("Method 1 Candidates", len(face_candidates))
        
    except Exception as e:
        print_result("Method 1 Error", str(e))
    
    # If no candidates from method 1, try method 2
    if not face_candidates:
        print("\n  Method 2: Edge-based Detection")
        try:
            edges = cv2.Canny(preprocessed, 50, 150)
            edge_pixels = np.sum(edges > 0)
            print_result("Edge Pixels Found", edge_pixels)
            
            # Save Method 2 edge image
            method2_edges_filename = f"{self.thesis_image_counter:03d}_method2_edges.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_edges_filename), edges)
            print_result("Method 2 Edges Saved", str(self.face_detection_output_dir / method2_edges_filename))
            
            kernel = np.ones((3, 3), np.uint8)
            dilated = cv2.dilate(edges, kernel, iterations=1)
            print_result("Edge Dilation", "Applied")
            
            # Save Method 2 dilated edges
            method2_dilated_filename = f"{self.thesis_image_counter:03d}_method2_dilated.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_dilated_filename), dilated)
            print_result("Method 2 Dilated Saved", str(self.face_detection_output_dir / method2_dilated_filename))
            
            contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            print_result("Edge Contours Found", len(contours))
            
            # Create visualization for Method 2
            method2_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
            
            valid_edge_contours = 0
            for contour in contours:
                x, y, w_cont, h_cont = cv2.boundingRect(contour)
                area = w_cont * h_cont
                img_area = preprocessed.shape[0] * preprocessed.shape[1]
                
                if 0.05 * img_area < area < 0.9 * img_area:
                    face_candidates.append((x, y, x+w_cont, y+h_cont))
                    valid_edge_contours += 1
                    print_result(f"Edge Contour {valid_edge_contours}", f"({x}, {y}, {x+w_cont}, {y+h_cont}) - Area: {area}")
                    # Draw valid rectangles
                    cv2.rectangle(method2_viz, (x, y), (x+w_cont, y+h_cont), (0, 255, 255), 3)
            
            # Sort by area (largest first)
            if face_candidates:
                face_candidates.sort(key=lambda rect: (rect[2]-rect[0])*(rect[3]-rect[1]), reverse=True)
                face_candidates = face_candidates[:3]  # Keep top 3
            
            # Save Method 2 visualization
            method2_viz_filename = f"{self.thesis_image_counter:03d}_method2_detection.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_viz_filename), method2_viz)
            print_result("Method 2 Visualization Saved", str(self.face_detection_output_dir / method2_viz_filename))
            
            print_result("Method 2 Candidates", len(face_candidates))
            
        except Exception as e:
            print_result("Method 2 Error", str(e))
    
    # Fallback method if no faces found
    if not face_candidates:
        print("\n  Method 3: Center Fallback Detection")
        center_x, center_y = w // 2, h // 2
        face_size = min(w, h) // 2
        fallback_rect = (center_x - face_size, center_y - face_size, 
                       center_x + face_size, center_y + face_size)
        face_candidates = [fallback_rect]
        print_result("Fallback Rectangle", f"({fallback_rect[0]}, {fallback_rect[1]}, {fallback_rect[2]}, {fallback_rect[3]})")
        
        # Create visualization for Method 3
        method3_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
        cv2.rectangle(method3_viz, (fallback_rect[0], fallback_rect[1]), 
                     (fallback_rect[2], fallback_rect[3]), (255, 0, 255), 3)
        cv2.putText(method3_viz, "FALLBACK CENTER", (fallback_rect[0], fallback_rect[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)
        
        # Save Method 3 visualization
        method3_viz_filename = f"{self.thesis_image_counter:03d}_method3_fallback.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method3_viz_filename), method3_viz)
        print_result("Method 3 Visualization Saved", str(self.face_detection_output_dir / method3_viz_filename))
    
    # Save final detection result with all candidates
    final_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
    for i, (x1, y1, x2, y2) in enumerate(face_candidates):
        color = (0, 255, 0) if i == 0 else (0, 165, 255)  # Green for primary, orange for others
        thickness = 3 if i == 0 else 2
        cv2.rectangle(final_viz, (x1, y1), (x2, y2), color, thickness)
        cv2.putText(final_viz, f"Face {i+1}", (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    
    final_detection_filename = f"{self.thesis_image_counter:03d}_final_detection.jpg"
    cv2.imwrite(str(self.face_detection_output_dir / final_detection_filename), final_viz)
    print_result("Final Detection Saved", str(self.face_detection_output_dir / final_detection_filename))
    
    print_result("Final Candidates Found", len(face_candidates))
    
    return face_candidates, preprocessed

# Update the method in the class
ElephantFaceIDDetailed.detect_elephant_face = detect_elephant_face
def detect_elephant_face(self, image):
    """Detect elephant face with detailed output"""
    print_step(4, "Detecting Elephant Face")
    
    preprocessed, gray = self.preprocess_image(image)
    h, w = preprocessed.shape
    print_result("Preprocessed Image Size", f"{w} x {h}")
    
    # Create directory for face detection outputs
    self.face_detection_output_dir = self.thesis_output_dir / "face_detection_steps"
    self.face_detection_output_dir.mkdir(exist_ok=True)
    
    face_candidates = []
    
    # Method 1: Thresholding and contour detection
    print("\n  Method 1: Contour-based Detection")
    try:
        _, binary = cv2.threshold(preprocessed, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        print_result("Otsu Threshold", "Applied")
        print_result("Binary Pixel Distribution", f"White: {np.sum(binary == 255)}, Black: {np.sum(binary == 0)}")
        
        # Save Method 1 binary image
        method1_binary_filename = f"{self.thesis_image_counter:03d}_method1_binary.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method1_binary_filename), binary)
        print_result("Method 1 Binary Saved", str(self.face_detection_output_dir / method1_binary_filename))
        
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print_result("Total Contours Found", len(contours))
        
        # Create visualization for Method 1
        method1_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(method1_viz, contours, -1, (0, 255, 0), 2)
        
        valid_contours = 0
        for contour in contours:
            x, y, w_cont, h_cont = cv2.boundingRect(contour)
            aspect_ratio = float(w_cont) / h_cont
            area_ratio = w_cont * h_cont / float(preprocessed.shape[0] * preprocessed.shape[1])
            
            if 0.5 <= aspect_ratio <= 2.0 and w_cont > 50 and h_cont > 50:
                if 0.1 <= area_ratio <= 0.9:
                    face_candidates.append((x, y, x+w_cont, y+h_cont))
                    valid_contours += 1
                    print_result(f"Valid Contour {valid_contours}", f"({x}, {y}, {x+w_cont}, {y+h_cont}) - AR: {aspect_ratio:.2f}, Area: {area_ratio:.3f}")
                    # Draw valid rectangles
                    cv2.rectangle(method1_viz, (x, y), (x+w_cont, y+h_cont), (255, 0, 0), 3)
        
        # Save Method 1 visualization
        method1_viz_filename = f"{self.thesis_image_counter:03d}_method1_contours.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method1_viz_filename), method1_viz)
        print_result("Method 1 Visualization Saved", str(self.face_detection_output_dir / method1_viz_filename))
        
        print_result("Method 1 Candidates", len(face_candidates))
        
    except Exception as e:
        print_result("Method 1 Error", str(e))
    
    # If no candidates from method 1, try method 2
    if not face_candidates:
        print("\n  Method 2: Edge-based Detection")
        try:
            edges = cv2.Canny(preprocessed, 50, 150)
            edge_pixels = np.sum(edges > 0)
            print_result("Edge Pixels Found", edge_pixels)
            
            # Save Method 2 edge image
            method2_edges_filename = f"{self.thesis_image_counter:03d}_method2_edges.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_edges_filename), edges)
            print_result("Method 2 Edges Saved", str(self.face_detection_output_dir / method2_edges_filename))
            
            kernel = np.ones((3, 3), np.uint8)
            dilated = cv2.dilate(edges, kernel, iterations=1)
            print_result("Edge Dilation", "Applied")
            
            # Save Method 2 dilated edges
            method2_dilated_filename = f"{self.thesis_image_counter:03d}_method2_dilated.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_dilated_filename), dilated)
            print_result("Method 2 Dilated Saved", str(self.face_detection_output_dir / method2_dilated_filename))
            
            contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            print_result("Edge Contours Found", len(contours))
            
            # Create visualization for Method 2
            method2_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
            
            valid_edge_contours = 0
            for contour in contours:
                x, y, w_cont, h_cont = cv2.boundingRect(contour)
                area = w_cont * h_cont
                img_area = preprocessed.shape[0] * preprocessed.shape[1]
                
                if 0.05 * img_area < area < 0.9 * img_area:
                    face_candidates.append((x, y, x+w_cont, y+h_cont))
                    valid_edge_contours += 1
                    print_result(f"Edge Contour {valid_edge_contours}", f"({x}, {y}, {x+w_cont}, {y+h_cont}) - Area: {area}")
                    # Draw valid rectangles
                    cv2.rectangle(method2_viz, (x, y), (x+w_cont, y+h_cont), (0, 255, 255), 3)
            
            # Sort by area (largest first)
            if face_candidates:
                face_candidates.sort(key=lambda rect: (rect[2]-rect[0])*(rect[3]-rect[1]), reverse=True)
                face_candidates = face_candidates[:3]  # Keep top 3
            
            # Save Method 2 visualization
            method2_viz_filename = f"{self.thesis_image_counter:03d}_method2_detection.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_viz_filename), method2_viz)
            print_result("Method 2 Visualization Saved", str(self.face_detection_output_dir / method2_viz_filename))
            
            print_result("Method 2 Candidates", len(face_candidates))
            
        except Exception as e:
            print_result("Method 2 Error", str(e))
    
    # Fallback method if no faces found
    if not face_candidates:
        print("\n  Method 3: Center Fallback Detection")
        center_x, center_y = w // 2, h // 2
        face_size = min(w, h) // 2
        fallback_rect = (center_x - face_size, center_y - face_size, 
                       center_x + face_size, center_y + face_size)
        face_candidates = [fallback_rect]
        print_result("Fallback Rectangle", f"({fallback_rect[0]}, {fallback_rect[1]}, {fallback_rect[2]}, {fallback_rect[3]})")
        
        # Create visualization for Method 3
        method3_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
        cv2.rectangle(method3_viz, (fallback_rect[0], fallback_rect[1]), 
                     (fallback_rect[2], fallback_rect[3]), (255, 0, 255), 3)
        cv2.putText(method3_viz, "FALLBACK CENTER", (fallback_rect[0], fallback_rect[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)
        
        # Save Method 3 visualization
        method3_viz_filename = f"{self.thesis_image_counter:03d}_method3_fallback.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method3_viz_filename), method3_viz)
        print_result("Method 3 Visualization Saved", str(self.face_detection_output_dir / method3_viz_filename))
    
    # Save final detection result with all candidates
    final_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
    for i, (x1, y1, x2, y2) in enumerate(face_candidates):
        color = (0, 255, 0) if i == 0 else (0, 165, 255)  # Green for primary, orange for others
        thickness = 3 if i == 0 else 2
        cv2.rectangle(final_viz, (x1, y1), (x2, y2), color, thickness)
        cv2.putText(final_viz, f"Face {i+1}", (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    
    final_detection_filename = f"{self.thesis_image_counter:03d}_final_detection.jpg"
    cv2.imwrite(str(self.face_detection_output_dir / final_detection_filename), final_viz)
    print_result("Final Detection Saved", str(self.face_detection_output_dir / final_detection_filename))
    
    print_result("Final Candidates Found", len(face_candidates))
    
    return face_candidates, preprocessed

# Update the method in the class
ElephantFaceIDDetailed.detect_elephant_face = detect_elephant_face
def detect_elephant_face(self, image):
    """Detect elephant face with detailed output"""
    print_step(4, "Detecting Elephant Face")
    
    preprocessed, gray = self.preprocess_image(image)
    h, w = preprocessed.shape
    print_result("Preprocessed Image Size", f"{w} x {h}")
    
    # Create directory for face detection outputs
    self.face_detection_output_dir = self.thesis_output_dir / "face_detection_steps"
    self.face_detection_output_dir.mkdir(exist_ok=True)
    
    face_candidates = []
    
    # Method 1: Thresholding and contour detection
    print("\n  Method 1: Contour-based Detection")
    try:
        _, binary = cv2.threshold(preprocessed, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        print_result("Otsu Threshold", "Applied")
        print_result("Binary Pixel Distribution", f"White: {np.sum(binary == 255)}, Black: {np.sum(binary == 0)}")
        
        # Save Method 1 binary image
        method1_binary_filename = f"{self.thesis_image_counter:03d}_method1_binary.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method1_binary_filename), binary)
        print_result("Method 1 Binary Saved", str(self.face_detection_output_dir / method1_binary_filename))
        
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print_result("Total Contours Found", len(contours))
        
        # Create visualization for Method 1
        method1_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(method1_viz, contours, -1, (0, 255, 0), 2)
        
        valid_contours = 0
        for contour in contours:
            x, y, w_cont, h_cont = cv2.boundingRect(contour)
            aspect_ratio = float(w_cont) / h_cont
            area_ratio = w_cont * h_cont / float(preprocessed.shape[0] * preprocessed.shape[1])
            
            if 0.5 <= aspect_ratio <= 2.0 and w_cont > 50 and h_cont > 50:
                if 0.1 <= area_ratio <= 0.9:
                    face_candidates.append((x, y, x+w_cont, y+h_cont))
                    valid_contours += 1
                    print_result(f"Valid Contour {valid_contours}", f"({x}, {y}, {x+w_cont}, {y+h_cont}) - AR: {aspect_ratio:.2f}, Area: {area_ratio:.3f}")
                    # Draw valid rectangles
                    cv2.rectangle(method1_viz, (x, y), (x+w_cont, y+h_cont), (255, 0, 0), 3)
        
        # Save Method 1 visualization
        method1_viz_filename = f"{self.thesis_image_counter:03d}_method1_contours.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method1_viz_filename), method1_viz)
        print_result("Method 1 Visualization Saved", str(self.face_detection_output_dir / method1_viz_filename))
        
        print_result("Method 1 Candidates", len(face_candidates))
        
    except Exception as e:
        print_result("Method 1 Error", str(e))
    
    # If no candidates from method 1, try method 2
    if not face_candidates:
        print("\n  Method 2: Edge-based Detection")
        try:
            edges = cv2.Canny(preprocessed, 50, 150)
            edge_pixels = np.sum(edges > 0)
            print_result("Edge Pixels Found", edge_pixels)
            
            # Save Method 2 edge image
            method2_edges_filename = f"{self.thesis_image_counter:03d}_method2_edges.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_edges_filename), edges)
            print_result("Method 2 Edges Saved", str(self.face_detection_output_dir / method2_edges_filename))
            
            kernel = np.ones((3, 3), np.uint8)
            dilated = cv2.dilate(edges, kernel, iterations=1)
            print_result("Edge Dilation", "Applied")
            
            # Save Method 2 dilated edges
            method2_dilated_filename = f"{self.thesis_image_counter:03d}_method2_dilated.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_dilated_filename), dilated)
            print_result("Method 2 Dilated Saved", str(self.face_detection_output_dir / method2_dilated_filename))
            
            contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            print_result("Edge Contours Found", len(contours))
            
            # Create visualization for Method 2
            method2_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
            
            valid_edge_contours = 0
            for contour in contours:
                x, y, w_cont, h_cont = cv2.boundingRect(contour)
                area = w_cont * h_cont
                img_area = preprocessed.shape[0] * preprocessed.shape[1]
                
                if 0.05 * img_area < area < 0.9 * img_area:
                    face_candidates.append((x, y, x+w_cont, y+h_cont))
                    valid_edge_contours += 1
                    print_result(f"Edge Contour {valid_edge_contours}", f"({x}, {y}, {x+w_cont}, {y+h_cont}) - Area: {area}")
                    # Draw valid rectangles
                    cv2.rectangle(method2_viz, (x, y), (x+w_cont, y+h_cont), (0, 255, 255), 3)
            
            # Sort by area (largest first)
            if face_candidates:
                face_candidates.sort(key=lambda rect: (rect[2]-rect[0])*(rect[3]-rect[1]), reverse=True)
                face_candidates = face_candidates[:3]  # Keep top 3
            
            # Save Method 2 visualization
            method2_viz_filename = f"{self.thesis_image_counter:03d}_method2_detection.jpg"
            cv2.imwrite(str(self.face_detection_output_dir / method2_viz_filename), method2_viz)
            print_result("Method 2 Visualization Saved", str(self.face_detection_output_dir / method2_viz_filename))
            
            print_result("Method 2 Candidates", len(face_candidates))
            
        except Exception as e:
            print_result("Method 2 Error", str(e))
    
    # Fallback method if no faces found
    if not face_candidates:
        print("\n  Method 3: Center Fallback Detection")
        center_x, center_y = w // 2, h // 2
        face_size = min(w, h) // 2
        fallback_rect = (center_x - face_size, center_y - face_size, 
                       center_x + face_size, center_y + face_size)
        face_candidates = [fallback_rect]
        print_result("Fallback Rectangle", f"({fallback_rect[0]}, {fallback_rect[1]}, {fallback_rect[2]}, {fallback_rect[3]})")
        
        # Create visualization for Method 3
        method3_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
        cv2.rectangle(method3_viz, (fallback_rect[0], fallback_rect[1]), 
                     (fallback_rect[2], fallback_rect[3]), (255, 0, 255), 3)
        cv2.putText(method3_viz, "FALLBACK CENTER", (fallback_rect[0], fallback_rect[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)
        
        # Save Method 3 visualization
        method3_viz_filename = f"{self.thesis_image_counter:03d}_method3_fallback.jpg"
        cv2.imwrite(str(self.face_detection_output_dir / method3_viz_filename), method3_viz)
        print_result("Method 3 Visualization Saved", str(self.face_detection_output_dir / method3_viz_filename))
    
    # Save final detection result with all candidates
    final_viz = cv2.cvtColor(preprocessed, cv2.COLOR_GRAY2BGR)
    for i, (x1, y1, x2, y2) in enumerate(face_candidates):
        color = (0, 255, 0) if i == 0 else (0, 165, 255)  # Green for primary, orange for others
        thickness = 3 if i == 0 else 2
        cv2.rectangle(final_viz, (x1, y1), (x2, y2), color, thickness)
        cv2.putText(final_viz, f"Face {i+1}", (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    
    final_detection_filename = f"{self.thesis_image_counter:03d}_final_detection.jpg"
    cv2.imwrite(str(self.face_detection_output_dir / final_detection_filename), final_viz)
    print_result("Final Detection Saved", str(self.face_detection_output_dir / final_detection_filename))
    
    print_result("Final Candidates Found", len(face_candidates))
    
    return face_candidates, preprocessed

# Update the method in the class
ElephantFaceIDDetailed.detect_elephant_face = detect_elephant_face

# Cell 7: Face Alignment with Thesis Comparison Images
def _create_alignment_comparison(self, before_img, after_img, image_name):
    """Create side-by-side comparison of before and after alignment"""
    # Resize before image to match after image size for fair comparison
    before_resized = cv2.resize(before_img, (224, 224))
    
    # Create side-by-side comparison
    h, w = after_img.shape[:2]
    comparison = np.zeros((h, w * 2, 3), dtype=np.uint8)
    
    # Convert grayscale to BGR if needed
    if len(before_resized.shape) == 2:
        before_resized = cv2.cvtColor(before_resized, cv2.COLOR_GRAY2BGR)
    if len(after_img.shape) == 2:
        after_img = cv2.cvtColor(after_img, cv2.COLOR_GRAY2BGR)
    
    # Place images side by side
    comparison[:, :w] = before_resized
    comparison[:, w:] = after_img
    
    # Add labels
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    color = (0, 255, 0)  # Green
    thickness = 2
    
    cv2.putText(comparison, "BEFORE Alignment", (10, 30), font, font_scale, color, thickness)
    cv2.putText(comparison, "AFTER Alignment", (w + 10, 30), font, font_scale, color, thickness)
    
    # Add dividing line
    cv2.line(comparison, (w, 0), (w, h), (255, 255, 255), 2)
    
    return comparison

def align_elephant_face(self, image, face_rect, image_name="unknown"):
    """Align elephant face with detailed output and thesis comparison images"""
    print_step(5, "Aligning Elephant Face")
    
    x1, y1, x2, y2 = face_rect
    h, w = image.shape[:2]
    
    print_result("Original Face Rect", f"({x1}, {y1}, {x2}, {y2})")
    print_result("Image Dimensions", f"{w} x {h}")
    
    # Ensure coordinates are within bounds
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)
    print_result("Bounded Face Rect", f"({x1}, {y1}, {x2}, {y2})")
    
    # Check validity
    if x2 <= x1 or y2 <= y1:
        print_result("Face Rectangle", "INVALID - Using zero array")
        return np.zeros((224, 224), dtype=np.uint8)
    
    try:
        face_img = image[y1:y2, x1:x2].copy()
        face_shape = face_img.shape
        print_result("Extracted Face Shape", face_shape)
        
        # THESIS COMPARISON: Save BEFORE alignment image
        self.thesis_image_counter += 1
        before_filename = f"{self.thesis_image_counter:03d}_{Path(image_name).stem}_before_alignment.jpg"
        before_path = self.alignment_comparison_dir / before_filename
        cv2.imwrite(str(before_path), face_img)
        print_result("THESIS - Before Alignment Saved", str(before_path))
        
    except Exception as e:
        print_result("Face Extraction Error", str(e))
        return np.zeros((224, 224), dtype=np.uint8)
    
    # Check minimum size
    if face_img.shape[0] < 10 or face_img.shape[1] < 10:
        print_result("Face Size", "TOO SMALL - Expanding region")
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
        size = max(50, max(x2-x1, y2-y1))
        
        new_x1 = max(0, center_x - size // 2)
        new_y1 = max(0, center_y - size // 2)
        new_x2 = min(w, center_x + size // 2)
        new_y2 = min(h, center_y + size // 2)
        
        print_result("Expanded Rectangle", f"({new_x1}, {new_y1}, {new_x2}, {new_y2})")
        
        try:
            face_img = image[new_y1:new_y2, new_x1:new_x2].copy()
            print_result("Expanded Face Shape", face_img.shape)
            
            # Update THESIS COMPARISON: Save updated BEFORE alignment image
            before_filename = f"{self.thesis_image_counter:03d}_{Path(image_name).stem}_before_alignment_expanded.jpg"
            before_path = self.alignment_comparison_dir / before_filename
            cv2.imwrite(str(before_path), face_img)
            print_result("THESIS - Before Alignment (Expanded) Saved", str(before_path))
            
        except Exception as e:
            print_result("Expanded Extraction Error", str(e))
            return np.zeros((224, 224), dtype=np.uint8)
    
    # Resize to standard size
    standard_size = (224, 224)
    try:
        aligned_face = cv2.resize(face_img, standard_size)
        print_result("Final Aligned Size", f"{aligned_face.shape[1]} x {aligned_face.shape[0]}")
        print_result("Alignment Status", "SUCCESS")
        
        # THESIS COMPARISON: Save AFTER alignment image
        after_filename = f"{self.thesis_image_counter:03d}_{Path(image_name).stem}_after_alignment.jpg"
        after_path = self.alignment_comparison_dir / after_filename
        cv2.imwrite(str(after_path), aligned_face)
        print_result("THESIS - After Alignment Saved", str(after_path))
        
        # Create side-by-side comparison image
        comparison_img = self._create_alignment_comparison(face_img, aligned_face, image_name)
        comparison_filename = f"{self.thesis_image_counter:03d}_{Path(image_name).stem}_alignment_comparison.jpg"
        comparison_path = self.alignment_comparison_dir / comparison_filename
        cv2.imwrite(str(comparison_path), comparison_img)
        print_result("THESIS - Alignment Comparison Saved", str(comparison_path))
        
        return aligned_face
    except Exception as e:
        print_result("Resize Error", str(e))
        return np.zeros(standard_size, dtype=np.uint8)

# Add methods to class
ElephantFaceIDDetailed._create_alignment_comparison = _create_alignment_comparison
ElephantFaceIDDetailed.align_elephant_face = align_elephant_face

print("Face alignment with thesis comparison methods defined!")

# Cell 8: View Augmentation with Thesis Comparison Images - Part 1
def _create_augmentation_comparison_grid(self, original, rotation_images, perspective_images, brightness_images, image_name):
    """Create a comprehensive grid showing original vs all augmentations"""
    try:
        # Calculate grid dimensions
        total_images = 1 + len(rotation_images) + len(perspective_images) + len(brightness_images)
        cols = 4  # 4 images per row
        rows = (total_images + cols - 1) // cols
        
        img_size = 224
        grid_img = np.zeros((rows * img_size, cols * img_size, 3), dtype=np.uint8)
        
        # Convert original to BGR if grayscale
        if len(original.shape) == 2:
            original_bgr = cv2.cvtColor(original, cv2.COLOR_GRAY2BGR)
        else:
            original_bgr = original.copy()
        
        # Place original image
        grid_img[0:img_size, 0:img_size] = original_bgr
        
        # Add label to original
        cv2.putText(grid_img, "ORIGINAL", (5, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        current_pos = 1
        
        # Add rotation images
        for rotated, angle in rotation_images:
            if current_pos >= total_images:
                break
            row = current_pos // cols
            col = current_pos % cols
            
            if len(rotated.shape) == 2:
                rotated_bgr = cv2.cvtColor(rotated, cv2.COLOR_GRAY2BGR)
            else:
                rotated_bgr = rotated.copy()
            
            y_start = row * img_size
            y_end = y_start + img_size
            x_start = col * img_size
            x_end = x_start + img_size
            
            grid_img[y_start:y_end, x_start:x_end] = rotated_bgr
            cv2.putText(grid_img, f"ROT {angle:+d}°", (x_start + 5, y_start + 25), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
            current_pos += 1
        
        # Add perspective images
        for perspective, view_name in perspective_images:
            if current_pos >= total_images:
                break
            row = current_pos // cols
            col = current_pos % cols
            
            if len(perspective.shape) == 2:
                perspective_bgr = cv2.cvtColor(perspective, cv2.COLOR_GRAY2BGR)
            else:
                perspective_bgr = perspective.copy()
            
            y_start = row * img_size
            y_end = y_start + img_size
            x_start = col * img_size
            x_end = x_start + img_size
            
            grid_img[y_start:y_end, x_start:x_end] = perspective_bgr
            cv2.putText(grid_img, view_name.upper(), (x_start + 5, y_start + 25), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 255), 1)
            current_pos += 1
        
        # Add brightness images
        for brightness, factor in brightness_images:
            if current_pos >= total_images:
                break
            row = current_pos // cols
            col = current_pos % cols
            
            if len(brightness.shape) == 2:
                brightness_bgr = cv2.cvtColor(brightness, cv2.COLOR_GRAY2BGR)
            else:
                brightness_bgr = brightness.copy()
            
            y_start = row * img_size
            y_end = y_start + img_size
            x_start = col * img_size
            x_end = x_start + img_size
            
            grid_img[y_start:y_end, x_start:x_end] = brightness_bgr
            cv2.putText(grid_img, f"BRIGHT {factor:.1f}", (x_start + 5, y_start + 25), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)
            current_pos += 1
        
        # Add grid lines
        for i in range(1, cols):
            cv2.line(grid_img, (i * img_size, 0), (i * img_size, rows * img_size), (255, 255, 255), 1)
        for i in range(1, rows):
            cv2.line(grid_img, (0, i * img_size), (cols * img_size, i * img_size), (255, 255, 255), 1)
        
        # Save comparison grid
        grid_filename = f"{self.thesis_image_counter:03d}_{Path(image_name).stem}_augmentation_comparison_grid.jpg"
        grid_path = self.augmentation_comparison_dir / grid_filename
        cv2.imwrite(str(grid_path), grid_img)
        print_result("THESIS - Augmentation Comparison Grid Saved", str(grid_path))
        
    except Exception as e:
        print_result("Augmentation Grid Creation Error", str(e))

# Add to class
ElephantFaceIDDetailed._create_augmentation_comparison_grid = _create_augmentation_comparison_grid

print("Augmentation comparison grid method defined!")

# Cell 9: View Augmentation with Thesis Comparison Images - Part 2
def create_augmented_views(self, aligned_face, image_name="unknown"):
    """Create augmented views with detailed output and thesis comparison images"""
    print_step(6, "Creating Augmented Views")
    
    original_shape = aligned_face.shape
    print_result("Original Face Shape", original_shape)
    
    # THESIS COMPARISON: Save BEFORE augmentation (original aligned face)
    original_filename = f"{self.thesis_image_counter:03d}_{Path(image_name).stem}_original_before_augmentation.jpg"
    original_path = self.augmentation_comparison_dir / original_filename
    cv2.imwrite(str(original_path), aligned_face)
    print_result("THESIS - Original Before Augmentation Saved", str(original_path))
    
    augmented_views = [aligned_face]
    
    # Rotation augmentations
    rotation_angles = [-15, -10, -5, 5, 10, 15]
    print_result("Rotation Angles", rotation_angles)
    
    successful_rotations = 0
    rotation_images = []
    for angle in rotation_angles:
        try:
            h, w = aligned_face.shape[:2]
            center = (w // 2, h // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated = cv2.warpAffine(aligned_face, rotation_matrix, (w, h))
            augmented_views.append(rotated)
            rotation_images.append((rotated, angle))
            successful_rotations += 1
            
            # THESIS COMPARISON: Save individual rotation
            rotation_filename = f"{self.thesis_image_counter:03d}_{Path(image_name).stem}_rotation_{angle:+03d}_degrees.jpg"
            rotation_path = self.augmentation_comparison_dir / rotation_filename
            cv2.imwrite(str(rotation_path), rotated)
            print_result(f"THESIS - Rotation {angle}° Saved", str(rotation_path))
            
        except Exception as e:
            print_result(f"Rotation {angle}° Error", str(e))
    
    print_result("Successful Rotations", successful_rotations)
    
    # Perspective transformations
    h, w = aligned_face.shape[:2]
    perspective_variations = [
        # top view
        (np.float32([[0, 0], [w, 0], [0, h], [w, h]]), 
         np.float32([[0, 10], [w, 0], [0, h-10], [w, h]]), "top_view"),
        # side view left
        (np.float32([[0, 0], [w, 0], [0, h], [w, h]]), 
         np.float32([[10, 0], [w-10, 0], [0, h], [w, h]]), "side_left"),
        # side view right
        (np.float32([[0, 0], [w, 0], [0, h], [w, h]]), 
         np.float32([[0, 0], [w-10, 0], [10, h], [w, h]]), "side_right"),
        # bottom view
        (np.float32([[0, 0], [w, 0], [0, h], [w, h]]), 
         np.float32([[0, 0], [w, 10], [0, h-10], [w, h]]), "bottom_view")
    ]
    
    successful_perspectives = 0
    perspective_images = []
    for i, (src_pts, dst_pts, view_name) in enumerate(perspective_variations):
        try:
            perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
            perspective_view = cv2.warpPerspective(aligned_face, perspective_matrix, (w, h))
            augmented_views.append(perspective_view)
            perspective_images.append((perspective_view, view_name))
            successful_perspectives += 1
            
            # THESIS COMPARISON: Save individual perspective
            perspective_filename = f"{self.thesis_image_counter:03d}_{Path(image_name).stem}_perspective_{view_name}.jpg"
            perspective_path = self.augmentation_comparison_dir / perspective_filename
            cv2.imwrite(str(perspective_path), perspective_view)
            print_result(f"THESIS - Perspective {view_name} Saved", str(perspective_path))
            
        except Exception as e:
            print_result(f"Perspective {i+1} Error", str(e))
    
    print_result("Successful Perspectives", successful_perspectives)
    
    # Brightness variations
    brightness_factors = [0.7, 1.3]
    successful_brightness = 0
    brightness_images = []
    for factor in brightness_factors:
        try:
            adjusted = cv2.convertScaleAbs(aligned_face, alpha=factor, beta=0)
            augmented_views.append(adjusted)
            brightness_images.append((adjusted, factor))
            successful_brightness += 1
            
            # THESIS COMPARISON: Save individual brightness adjustment
            brightness_filename = f"{self.thesis_image_counter:03d}_{Path(image_name).stem}_brightness_{factor:.1f}.jpg"
            brightness_path = self.augmentation_comparison_dir / brightness_filename
            cv2.imwrite(str(brightness_path), adjusted)
            print_result(f"THESIS - Brightness {factor} Saved", str(brightness_path))
            
        except Exception as e:
            print_result(f"Brightness {factor} Error", str(e))
    
    print_result("Successful Brightness", successful_brightness)
    print_result("Total Augmented Views", len(augmented_views))
    
    # THESIS COMPARISON: Create comprehensive comparison grid
    self._create_augmentation_comparison_grid(aligned_face, rotation_images, perspective_images, brightness_images, image_name)
    
    return augmented_views

# Add to class
ElephantFaceIDDetailed.create_augmented_views = create_augmented_views

print("View augmentation with thesis comparison methods defined!")

# Cell 10: Feature Extraction Methods
def extract_texture_features(self, aligned_face):
    """Extract texture features with detailed output"""
    print_step(7, "Extracting Texture Features")
    
    # Ensure grayscale
    if len(aligned_face.shape) == 3:
        gray = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2GRAY)
        print_result("Color Conversion", "BGR to Grayscale")
    else:
        gray = aligned_face.copy()
        print_result("Input Format", "Already Grayscale")
    
    print_result("Gray Image Shape", gray.shape)
    print_result("Gray Pixel Range", f"{gray.min()} - {gray.max()}")
    
    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    print_result("CLAHE Applied", "Enhanced texture contrast")
    
    features = []
    
    # 1. Edge features
    edges = cv2.Canny(enhanced, 50, 150)
    edge_density = np.sum(edges > 0) / (edges.shape[0] * edges.shape[1])
    features.append(edge_density)
    print_result("Edge Density", f"{edge_density:.4f}")
    
    # 2. Gradient features
    gx = cv2.Sobel(enhanced, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(enhanced, cv2.CV_32F, 0, 1, ksize=3)
    mag = cv2.magnitude(gx, gy)
    
    grad_mean = np.mean(mag)
    grad_std = np.std(mag)
    grad_median = np.median(mag)
    
    features.extend([grad_mean, grad_std, grad_median])
    print_result("Gradient Mean", f"{grad_mean:.4f}")
    print_result("Gradient Std", f"{grad_std:.4f}")
    print_result("Gradient Median", f"{grad_median:.4f}")
    
    # 3. Local Binary Pattern
    h, w = enhanced.shape
    lbp = np.zeros((h-2, w-2), dtype=np.uint8)
    
    for i in range(1, h-1):
        for j in range(1, w-1):
            center = enhanced[i, j]
            code = 0
            neighbors = [
                enhanced[i-1, j],  # top
                enhanced[i, j+1],  # right
                enhanced[i+1, j],  # bottom
                enhanced[i, j-1]   # left
            ]
            
            for k, neighbor in enumerate(neighbors):
                if neighbor >= center:
                    code |= (1 << k)
                    
            lbp[i-1, j-1] = code
    
    hist, _ = np.histogram(lbp.ravel(), bins=16, range=(0, 16))
    hist = hist.astype("float")
    hist = hist / (hist.sum() + 1e-7)
    features.extend(hist)
    print_result("LBP Histogram Features", f"{len(hist)} bins")
    print_result("LBP Histogram Sum", f"{np.sum(hist):.4f}")
    
    # 4. Global intensity statistics
    intensity_stats = [
        np.mean(enhanced),
        np.std(enhanced),
        np.percentile(enhanced, 25),
        np.percentile(enhanced, 75)
    ]
    features.extend(intensity_stats)
    print_result("Intensity Mean", f"{intensity_stats[0]:.4f}")
    print_result("Intensity Std", f"{intensity_stats[1]:.4f}")
    print_result("Intensity Q1", f"{intensity_stats[2]:.4f}")
    print_result("Intensity Q3", f"{intensity_stats[3]:.4f}")
    
    feature_array = np.array(features)
    print_result("Total Texture Features", len(feature_array))
    print_result("Feature Range", f"{feature_array.min():.4f} - {feature_array.max():.4f}")
    
    return feature_array

def extract_anatomical_features(self, aligned_face):
    """Extract anatomical features with detailed output"""
    print_step(8, "Extracting Anatomical Features")
    
    if len(aligned_face.shape) == 3:
        gray = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2GRAY)
    else:
        gray = aligned_face.copy()
    
    print_result("Input Shape", gray.shape)
    
    # Get edges
    edges = cv2.Canny(gray, 50, 150)
    total_edge_pixels = np.sum(edges > 0)
    print_result("Total Edge Pixels", total_edge_pixels)
    
    # Divide into grid
    h, w = gray.shape
    grid_size = 4
    cell_h, cell_w = h // grid_size, w // grid_size
    print_result("Grid Size", f"{grid_size}x{grid_size}")
    print_result("Cell Dimensions", f"{cell_w} x {cell_h}")
    
    features = []
    
    # Extract features for each grid cell
    for i in range(grid_size):
        for j in range(grid_size):
            y_start = i * cell_h
            y_end = min((i + 1) * cell_h, h)
            x_start = j * cell_w
            x_end = min((j + 1) * cell_w, w)
            
            # Get cell from edges
            cell_edges = edges[y_start:y_end, x_start:x_end]
            density = np.sum(cell_edges > 0) / (cell_edges.shape[0] * cell_edges.shape[1])
            features.append(density)
            
            # Cell statistics
            cell_gray = gray[y_start:y_end, x_start:x_end]
            features.append(np.mean(cell_gray))
            features.append(np.std(cell_gray))
    
    print_result("Grid Features Extracted", f"{len(features)} features from {grid_size*grid_size} cells")
    
    # Extract regional features
    center_y, center_x = h // 2, w // 2
    center_size = min(h, w) // 4
    
    # 1. Central region (trunk area)
    center_region = gray[
        center_y - center_size:center_y + center_size,
        center_x - center_size:center_x + center_size
    ]
    center_mean = np.mean(center_region)
    features.append(center_mean)
    print_result("Center Region Mean", f"{center_mean:.4f}")
    
    # 2. Left ear region
    left_ear = gray[:h//3, :w//3]
    left_ear_mean = np.mean(left_ear)
    features.append(left_ear_mean)
    print_result("Left Ear Mean", f"{left_ear_mean:.4f}")
    
    # 3. Right ear region
    right_ear = gray[:h//3, 2*w//3:]
    right_ear_mean = np.mean(right_ear)
    features.append(right_ear_mean)
    print_result("Right Ear Mean", f"{right_ear_mean:.4f}")
    
    # 4. Left eye region
    left_eye = gray[h//3:(2*h//3), w//4:(2*w//4)]
    left_eye_mean = np.mean(left_eye)
    features.append(left_eye_mean)
    print_result("Left Eye Mean", f"{left_eye_mean:.4f}")
    
    # 5. Right eye region
    right_eye = gray[h//3:(2*h//3), (2*w//4):(3*w//4)]
    right_eye_mean = np.mean(right_eye)
    features.append(right_eye_mean)
    print_result("Right Eye Mean", f"{right_eye_mean:.4f}")
    
    feature_array = np.array(features)
    print_result("Total Anatomical Features", len(feature_array))
    print_result("Feature Range", f"{feature_array.min():.4f} - {feature_array.max():.4f}")
    
    return feature_array

def extract_all_features(self, image, face_rect, image_name="unknown"):
    """Extract all features with detailed output"""
    print_step(9, "Extracting All Features")
    
    # Get aligned face (with thesis comparison images)
    aligned_face = self.align_elephant_face(image, face_rect, image_name)
    
    # Create augmented views (with thesis comparison images)
    augmented_views = self.create_augmented_views(aligned_face, image_name)
    
    # Extract features from each view
    all_features = []
    
    print_result("Processing Views", f"{len(augmented_views)} augmented views")
    
    for i, view in enumerate(augmented_views):
        print(f"\n  Processing View {i+1}/{len(augmented_views)}")
        try:
            texture_features = self.extract_texture_features(view)
            anatomical_features = self.extract_anatomical_features(view)
            
            view_features = np.hstack([texture_features, anatomical_features])
            all_features.append(view_features)
            print_result(f"View {i+1} Features", f"{len(view_features)} features extracted")
            
        except Exception as e:
            print_result(f"View {i+1} Error", str(e))
            continue
    
    if not all_features:
        raise ValueError("Failed to extract features from all views")
    
    # Calculate mean features across all views
    mean_features = np.mean(all_features, axis=0)
    
    print_result("Feature Aggregation", "Mean across all views")
    print_result("Final Feature Count", len(mean_features))
    print_result("Feature Statistics", f"Mean: {np.mean(mean_features):.4f}, Std: {np.std(mean_features):.4f}")
    
    return mean_features, aligned_face

# Add methods to class
ElephantFaceIDDetailed.extract_texture_features = extract_texture_features
ElephantFaceIDDetailed.extract_anatomical_features = extract_anatomical_features
ElephantFaceIDDetailed.extract_all_features = extract_all_features

print("Feature extraction methods defined!")

# Cell 11: Embedding and ID Generation Methods
def generate_embedding(self, features):
    """Generate embedding with detailed output"""
    print_step(10, "Generating Feature Embedding")
    
    # Clean features
    features_clean = np.nan_to_num(features)
    nan_count = np.sum(np.isnan(features))
    inf_count = np.sum(np.isinf(features))
    
    print_result("Original Features Shape", features.shape)
    print_result("NaN Values Found", nan_count)
    print_result("Inf Values Found", inf_count)
    print_result("Cleaned Features Range", f"{features_clean.min():.4f} - {features_clean.max():.4f}")
    
    # Check if we have trained models
    if self.scaler is None or self.pca is None:
        # Collect features for training
        if len(self.collected_features) < 5:
            self.collected_features.append(features_clean)
            print_result("Features Collection", f"{len(self.collected_features)}/5 samples collected")
        
        # Use hash-based embedding temporarily
        print_result("Embedding Method", "Hash-based (temporary)")
        feature_bytes = features_clean.tobytes()
        feature_hash = hashlib.md5(feature_bytes).digest()
        temp_embedding = np.array([b for b in feature_hash], dtype=np.float32)
        print_result("Hash Embedding Size", len(temp_embedding))
        return temp_embedding
    
    # Use trained models
    try:
        X_scaled = self.scaler.transform(features_clean.reshape(1, -1))
        embedding = self.pca.transform(X_scaled).flatten()
        
        print_result("Embedding Method", "PCA-based")
        print_result("Scaling Applied", "StandardScaler normalization")
        print_result("PCA Embedding Size", len(embedding))
        print_result("Embedding Range", f"{embedding.min():.4f} - {embedding.max():.4f}")
        
        return embedding
        
    except ValueError as e:
        print_result("PCA Error", str(e))
        print_result("Fallback Method", "Hash-based embedding")
        
        # Reset and use hash
        feature_bytes = features_clean.tobytes()
        feature_hash = hashlib.md5(feature_bytes).digest()
        temp_embedding = np.array([b for b in feature_hash], dtype=np.float32)
        return temp_embedding

def generate_unique_id(self, embedding):
    """Generate unique ID with detailed output"""
    print_step(11, "Generating Unique ID")
    
    # Convert embedding to bytes
    embedding_bytes = embedding.tobytes()
    print_result("Embedding Bytes Size", len(embedding_bytes))
    
    # Create hash
    h = hashlib.sha256(embedding_bytes)
    hash_hex = h.hexdigest()
    print_result("SHA-256 Hash", hash_hex[:32] + "...")
    
    # Generate stable ID
    unique_id = hash_hex[:16]
    formatted_id = f"ELE-{unique_id[:4]}-{unique_id[4:8]}-{unique_id[8:12]}-{unique_id[12:16]}"
    
    print_result("Raw ID", unique_id)
    print_result("Formatted ID", formatted_id)
    
    return formatted_id

# Add methods to class
ElephantFaceIDDetailed.generate_embedding = generate_embedding
ElephantFaceIDDetailed.generate_unique_id = generate_unique_id

print("Embedding and ID generation methods defined!")

# Cell 12: Matching and Database Methods
def match_embeddings(self, embedding1, embedding2):
    """Match embeddings with detailed output"""
    print_step(12, "Matching Embeddings")
    
    # Clean embeddings
    emb1 = np.nan_to_num(embedding1)
    emb2 = np.nan_to_num(embedding2)
    
    print_result("Embedding 1 Shape", emb1.shape)
    print_result("Embedding 2 Shape", emb2.shape)
    
    # Check compatibility
    if emb1.shape != emb2.shape:
        print_result("Shape Compatibility", "MISMATCH")
        return 0.0, False
    
    print_result("Shape Compatibility", "OK")
    
    # Calculate similarity metrics
    try:
        # Cosine similarity
        cosine_sim = 1 - cosine(emb1, emb2)
        print_result("Cosine Similarity", f"{cosine_sim:.4f}")
    except:
        cosine_sim = 0.0
        print_result("Cosine Similarity", "CALCULATION ERROR")
    
    try:
        # Euclidean similarity
        euclidean_dist = euclidean(emb1, emb2)
        max_dist = np.sqrt(len(emb1)) * 2
        euclidean_sim = 1 - (euclidean_dist / max_dist)
        print_result("Euclidean Distance", f"{euclidean_dist:.4f}")
        print_result("Euclidean Similarity", f"{euclidean_sim:.4f}")
    except:
        euclidean_sim = 0.0
        print_result("Euclidean Similarity", "CALCULATION ERROR")
    
    try:
        # Angular similarity
        norm1 = np.linalg.norm(emb1)
        norm2 = np.linalg.norm(emb2)
        if norm1 > 0 and norm2 > 0:
            angular_sim = np.dot(emb1, emb2) / (norm1 * norm2)
            angular_sim = (angular_sim + 1) / 2  # Scale to [0, 1]
        else:
            angular_sim = 0.0
        print_result("Angular Similarity", f"{angular_sim:.4f}")
    except:
        angular_sim = 0.0
        print_result("Angular Similarity", "CALCULATION ERROR")
    
    # Combined similarity
    similarity = 0.6 * cosine_sim + 0.2 * euclidean_sim + 0.2 * angular_sim
    is_match = similarity >= self.match_threshold
    
    print_result("Combined Similarity", f"{similarity:.4f}")
    print_result("Match Threshold", f"{self.match_threshold:.4f}")
    print_result("Is Match", "YES" if is_match else "NO")
    
    return similarity, is_match

def find_best_match(self, embedding):
    """Find best match with detailed output"""
    print_step(13, "Finding Best Match in Database")
    
    if not self.elephants_db:
        print_result("Database Status", "EMPTY")
        return None, 0.0, []
    
    print_result("Database Elephants", len(self.elephants_db))
    
    all_similarities = []
    best_id = None
    best_similarity = 0
    
    # Compare against all elephants
    for elephant_id, known_embeddings in self.elephants_db.items():
        print(f"\n  Comparing against {elephant_id} ({len(known_embeddings)} embeddings)")
        
        elephant_similarity = 0
        
        for j, known_emb in enumerate(known_embeddings):
            similarity, is_match = self.match_embeddings(embedding, known_emb)
            all_similarities.append((elephant_id, similarity))
            
            print(f"    Embedding {j+1}: {similarity:.4f}")
            
            if similarity > elephant_similarity:
                elephant_similarity = similarity
        
        print_result(f"{elephant_id} Best Similarity", f"{elephant_similarity:.4f}")
        
        if elephant_similarity > best_similarity:
            best_similarity = elephant_similarity
            best_id = elephant_id
    
    # Sort similarities
    all_similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Final decision
    is_match = best_similarity >= self.match_threshold
    matched_id = best_id if is_match else None
    
    print_result("Best Match ID", matched_id or "NO MATCH")
    print_result("Best Similarity", f"{best_similarity:.4f}")
    print_result("Match Decision", "MATCH" if is_match else "NEW ELEPHANT")
    
    return matched_id, best_similarity, all_similarities

def _update_database(self, final_id, matched_id, unique_id, embedding):
    """Update database with detailed output"""
    print_step(14, "Updating Database")
    
    try:
        if matched_id is None:
            # New elephant
            self.elephants_db[unique_id] = [embedding]
            print_result("Database Action", "ADDED NEW ELEPHANT")
            print_result("New Elephant ID", unique_id)
        else:
            # Existing elephant
            self.elephants_db[matched_id].append(embedding)
            print_result("Database Action", "UPDATED EXISTING ELEPHANT")
            print_result("Updated Elephant ID", matched_id)
            print_result("Total Embeddings", len(self.elephants_db[matched_id]))
        
        # Save database
        with open(self.db_path, 'wb') as f:
            pickle.dump(self.elephants_db, f)
        print_result("Database Save", "SUCCESS")
        print_result("Total Elephants", len(self.elephants_db))
        
    except Exception as e:
        print_result("Database Update Error", str(e))

# Add methods to class
ElephantFaceIDDetailed.match_embeddings = match_embeddings
ElephantFaceIDDetailed.find_best_match = find_best_match
ElephantFaceIDDetailed._update_database = _update_database

print("Matching and database methods defined!")

# Cell 13: Image Processing and Dataset Methods
def process_image(self, image_path):
    """Process single image with detailed output"""
    print_section_header(f"PROCESSING IMAGE: {Path(image_path).name}")
    
    # Check file existence
    image_path = Path(image_path)
    if not image_path.exists():
        print_result("File Status", "NOT FOUND")
        return None
    
    print_result("File Status", "FOUND")
    print_result("File Size", f"{image_path.stat().st_size / 1024:.1f} KB")
    
    # Load image
    image = cv2.imread(str(image_path))
    if image is None:
        print_result("Image Loading", "FAILED")
        return None
    
    print_result("Image Loading", "SUCCESS")
    print_result("Image Shape", f"{image.shape[1]} x {image.shape[0]} x {image.shape[2]}")
    print_result("Image Size", f"{image.shape[0] * image.shape[1]} pixels")
    
    # Detect face
    face_rects, preprocessed = self.detect_elephant_face(image)
    if not face_rects:
        print_result("Face Detection", "NO FACES FOUND")
        return None
    
    # Select largest face
    face_rect = max(face_rects, key=lambda rect: (rect[2]-rect[0])*(rect[3]-rect[1]))
    face_area = (face_rect[2]-face_rect[0]) * (face_rect[3]-face_rect[1])
    print_result("Selected Face", f"Area: {face_area} pixels")
    print_result("Face Rectangle", f"({face_rect[0]}, {face_rect[1]}, {face_rect[2]}, {face_rect[3]})")
    
    # Extract features
    features, aligned_face = self.extract_all_features(preprocessed, face_rect, image_path.name)
    
    # Generate embedding
    embedding = self.generate_embedding(features)
    
    # Generate ID
    unique_id = self.generate_unique_id(embedding)
    
    # Find matches
    matched_id, max_similarity, all_similarities = self.find_best_match(embedding)
    
    # Final ID assignment
    final_id = matched_id if matched_id else unique_id
    
    # Create result
    result = {
        'image_path': str(image_path),
        'face_rect': face_rect,
        'unique_id': final_id,
        'is_new': matched_id is None,
        'similarity': max_similarity if matched_id else 0.0,
        'embedding': embedding,
        'aligned_face': aligned_face,
        'features': features,
        'potential_matches': all_similarities[:5]
    }
    
    # Update database
    self._update_database(final_id, matched_id, unique_id, embedding)
    
    # Display final results
    print_section_header("FINAL IDENTIFICATION RESULT")
    print_result("ELEPHANT ID", final_id)
    print_result("STATUS", "EXISTING MATCH" if matched_id else "NEW ELEPHANT")
    if matched_id:
        print_result("MATCH CONFIDENCE", f"{max_similarity:.4f}")
    
    if all_similarities:
        print("\n  Top Potential Matches:")
        for i, (eid, sim) in enumerate(all_similarities[:3], 1):
            print(f"    {i}. {eid}: {sim:.4f}")
    
    # Add to processed images
    self.all_processed_images.append({
        'image_path': str(image_path),
        'unique_id': final_id,
        'embedding': embedding
    })
    
    return result

def merge_similar_elephants(self, similarity_threshold=0.50):
    """Merge similar elephants with detailed output"""
    print_step(16, "Analyzing Database for Duplicate IDs")
    
    if len(self.elephants_db) <= 1:
        print_result("Merge Analysis", "NOT ENOUGH ELEPHANTS")
        return 0
    
    print_result("Elephants to Analyze", len(self.elephants_db))
    print_result("Similarity Threshold", similarity_threshold)
    
    # Find potential merges
    merges = []
    elephant_ids = list(self.elephants_db.keys())
    
    comparisons_made = 0
    for i in range(len(elephant_ids)):
        for j in range(i+1, len(elephant_ids)):
            id1 = elephant_ids[i]
            id2 = elephant_ids[j]
            
            embeddings1 = self.elephants_db[id1]
            embeddings2 = self.elephants_db[id2]
            
            best_similarity = 0
            
            for emb1 in embeddings1:
                for emb2 in embeddings2:
                    try:
                        similarity, _ = self.match_embeddings(emb1, emb2)
                        best_similarity = max(best_similarity, similarity)
                    except:
                        continue
            
            comparisons_made += 1
            
            if best_similarity >= similarity_threshold:
                merges.append((id1, id2, best_similarity))
                print_result(f"Merge Candidate", f"{id1} ↔ {id2} (similarity: {best_similarity:.4f})")
    
    print_result("Comparisons Made", comparisons_made)
    print_result("Merge Candidates Found", len(merges))
    
    # Sort by similarity
    merges.sort(key=lambda x: x[2], reverse=True)
    
    # Process merges
    merged_count = 0
    already_merged = set()
    
    for id1, id2, similarity in merges:
        if id1 not in already_merged and id2 not in already_merged:
            # Keep the older ID
            keep_id = min(id1, id2, key=lambda x: int(x.split('-')[1], 16))
            merge_id = id2 if keep_id == id1 else id1
            
            print_result(f"Merge Action", f"Merging {merge_id} → {keep_id}")
            
            # Merge embeddings
            self.elephants_db[keep_id].extend(self.elephants_db[merge_id])
            del self.elephants_db[merge_id]
            
            already_merged.add(merge_id)
            merged_count += 1
    
    # Save updated database
    if merged_count > 0:
        with open(self.db_path, 'wb') as f:
            pickle.dump(self.elephants_db, f)
        print_result("Database Updated", f"Merged {merged_count} elephants")
    
    print_result("Final Unique Elephants", len(self.elephants_db))
    
    return merged_count

def process_dataset(self, dataset_path=None):
    """Process dataset with detailed output"""
    print_section_header("PROCESSING DATASET")
    
    if dataset_path is None:
        dataset_path = self.dataset_path
    
    dataset_path = Path(dataset_path)
    
    # Validate dataset path
    if not dataset_path.exists():
        print_result("Dataset Path", "NOT FOUND")
        return []
    
    print_result("Dataset Path", str(dataset_path))
    
    # Find images
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_paths = []
    
    for ext in image_extensions:
        found = list(dataset_path.glob(f'**/*{ext}'))
        image_paths.extend(found)
        print_result(f"{ext.upper()} Files", len(found))
    
    print_result("Total Images Found", len(image_paths))
    
    if not image_paths:
        print_result("Processing Status", "NO IMAGES TO PROCESS")
        return []
    
    # Process each image
    processed_results = []
    successful_processing = 0
    
    for idx, img_path in enumerate(image_paths):
        print(f"\n{'='*20} IMAGE {idx+1}/{len(image_paths)} {'='*20}")
        
        try:
            result = self.process_image(img_path)
            if result:
                processed_results.append(result)
                successful_processing += 1
                print_result("Processing", "SUCCESS")
            else:
                print_result("Processing", "FAILED")
        except Exception as e:
            print_result("Processing Error", str(e))
    
    print_section_header("DATASET PROCESSING COMPLETE")
    print_result("Images Processed", f"{successful_processing}/{len(image_paths)}")
    print_result("Success Rate", f"{(successful_processing/len(image_paths)*100):.1f}%")
    
    # Perform merging
    print_step(15, "Performing Elephant ID Merging")
    merge_count = self.merge_similar_elephants(similarity_threshold=0.50)
    print_result("Elephants Merged", merge_count)
    
    return processed_results

# Add methods to class
ElephantFaceIDDetailed.process_image = process_image
ElephantFaceIDDetailed.merge_similar_elephants = merge_similar_elephants
ElephantFaceIDDetailed.process_dataset = process_dataset

print("Image processing and dataset methods defined!")

# Cell 14: NFT Generator Class
class ElephantNFTGeneratorDetailed:
    def __init__(self, blockchain_network="rinkeby", contract_address=None):
        """Initialize NFT generator with detailed output"""
        print_section_header("INITIALIZING NFT GENERATOR")
        
        self.nft_output_dir = Path("elephant_nfts")
        self.nft_output_dir.mkdir(exist_ok=True)
        
        self.metadata_dir = self.nft_output_dir / "metadata"
        self.metadata_dir.mkdir(exist_ok=True)
        
        self.images_dir = self.nft_output_dir / "images"
        self.images_dir.mkdir(exist_ok=True)
        
        print_result("NFT Output Directory", self.nft_output_dir)
        print_result("Metadata Directory", self.metadata_dir)
        print_result("Images Directory", self.images_dir)
        
        # Color palettes
        self.color_palettes = {
            'savanna': [(233, 221, 199), (210, 180, 140), (85, 107, 47), (139, 69, 19), (160, 82, 45)],
            'forest': [(34, 139, 34), (0, 100, 0), (107, 142, 35), (139, 115, 85), (110, 38, 14)],
            'sunset': [(255, 215, 0), (255, 165, 0), (255, 69, 0), (139, 0, 0), (85, 26, 139)],
            'river': [(65, 105, 225), (30, 144, 255), (135, 206, 250), (32, 178, 170), (46, 139, 87)]
        }
        
        print_result("Color Palettes Available", len(self.color_palettes))
        for name, colors in self.color_palettes.items():
            print(f"    {name}: {len(colors)} colors")
        
        self.blockchain_network = blockchain_network
        self.contract_address = contract_address
        
        print_result("Blockchain Network", blockchain_network)
        print_result("Contract Address", contract_address or "Not provided")
        print_result("Blockchain Connection", "OFFLINE MODE")
    
    def create_nft_image(self, elephant_id, face_image=None, custom_photo=None, width=1024, height=1024):
        """Create NFT image with detailed output"""
        print_section_header(f"CREATING NFT IMAGE FOR {elephant_id}")
        
        print_result("Canvas Size", f"{width} x {height} pixels")
        print_result("Face Image Provided", "Yes" if face_image is not None else "No")
        print_result("Custom Photo Provided", "Yes" if custom_photo is not None else "No")
        
        # Create basic NFT image (simplified for demo)
        nft_image = Image.new('RGB', (width, height), color=(100, 150, 200))
        draw = ImageDraw.Draw(nft_image)
        
        # Add elephant ID text
        try:
            font_size = width // 20
            font = ImageFont.load_default()
            print_result("Font", "Default system font")
        except:
            font = ImageFont.load_default()
            print_result("Font", "Default system font")
        
        # Add ID text
        text_position = (width // 20, height // 20)
        draw.text(text_position, f"Elephant ID: {elephant_id}", fill=(255, 255, 255), font=font)
        
        # Convert and save
        filename = f"{elephant_id.replace('-', '_')}.jpg"
        image_path = self.images_dir / filename
        nft_image.save(image_path, quality=95)
        
        file_size = image_path.stat().st_size / 1024
        print_result("File Saved", str(image_path))
        print_result("File Size", f"{file_size:.1f} KB")
        
        return image_path
    
    def create_metadata(self, elephant_id, image_path, attributes=None):
        """Create metadata with detailed output"""
        print_section_header(f"CREATING METADATA FOR {elephant_id}")
        
        if attributes is None:
            attributes = []
        
        # Extract attributes from ID
        id_parts = elephant_id.split('-')
        id_hash = id_parts[1] if len(id_parts) > 1 else "0000"
        rarity_score = int(id_hash, 16) % 100
        
        print_result("ID Hash Component", id_hash)
        print_result("Rarity Score", rarity_score)
        
        # Base attributes
        base_attributes = [
            {"trait_type": "Species", "value": "African Elephant"},
            {"trait_type": "Conservation Status", "value": "Protected"},
            {"trait_type": "Rarity", "value": rarity_score},
            {"trait_type": "ID Hash", "value": id_hash}
        ]
        
        all_attributes = base_attributes + attributes
        print_result("Total Attributes", len(all_attributes))
        
        # Create metadata
        metadata = {
            "name": f"Elephant {elephant_id}",
            "description": f"This NFT represents a unique elephant with ID {elephant_id}. Each purchase supports elephant conservation efforts.",
            "image": str(image_path),
            "external_url": f"https://elephant-conservation.org/elephant/{elephant_id}",
            "attributes": all_attributes
        }
        
        # Save metadata
        filename = f"{elephant_id.replace('-', '_')}_metadata.json"
        metadata_path = self.metadata_dir / filename
        
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        file_size = metadata_path.stat().st_size
        print_result("Metadata File", str(metadata_path))
        print_result("Metadata Size", f"{file_size} bytes")
        
        return metadata_path
    
    def generate_nft_from_elephant_id(self, elephant_id, face_image=None, custom_photo=None, wallet_address=None, upload_to_ipfs=False):
        """Generate complete NFT with detailed output"""
        print_section_header(f"GENERATING COMPLETE NFT FOR {elephant_id}")
        
        start_time = time.time()
        
        # Create image
        print_step(1, "Creating NFT Artwork")
        image_path = self.create_nft_image(elephant_id, face_image, custom_photo)
        
        # Create metadata
        print_step(2, "Creating NFT Metadata")
        attributes = []
        if face_image is not None:
            attributes.append({"trait_type": "Has Face Image", "value": "Yes"})
        
        metadata_path = self.create_metadata(elephant_id, str(image_path), attributes)
        
        # Compile results
        nft_info = {
            "elephant_id": elephant_id,
            "image_path": str(image_path),
            "metadata_path": str(metadata_path),
            "image_ipfs_hash": None,
            "metadata_ipfs_hash": None,
            "transaction_hash": None,
            "created_at": datetime.now().isoformat()
        }
        
        # Save NFT info
        info_path = self.nft_output_dir / f"{elephant_id.replace('-', '_')}_nft_info.json"
        with open(info_path, 'w') as f:
            json.dump(nft_info, f, indent=2)
        
        generation_time = time.time() - start_time
        print_result("NFT Generation Time", f"{generation_time:.2f} seconds")
        print_result("NFT Info Saved", str(info_path))
        
        return nft_info

print("NFT Generator class defined!")

# Cell 15: Main Execution Function
def main():
    """Main function with comprehensive output"""
    print_section_header("ELEPHANT FACE ID & NFT SYSTEM - COMPLETE PROCESSING")
    
    # Initialize system
    elephant_id_system = ElephantFaceIDDetailed(dataset_path="D:\\Downloads\\Elephants")
    
    # Check dataset
    dataset_path = Path("D:\\Downloads\\Elephants")
    if dataset_path.exists():
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
        image_files = []
        for ext in image_extensions:
            image_files.extend(list(dataset_path.glob(f'**/*{ext}')))
        print_result("Dataset Images Found", len(image_files))
    else:
        print_result("Dataset Status", "PATH NOT FOUND - Using demo mode")
        # Create sample directory for demo
        sample_dir = Path("./elephant_samples")
        sample_dir.mkdir(exist_ok=True)
        elephant_id_system.dataset_path = sample_dir
    
    # Process dataset
    results = elephant_id_system.process_dataset()
    
    # Initialize NFT generator
    nft_generator = ElephantNFTGeneratorDetailed()
    
    # Generate NFTs for each unique elephant
    print_section_header("GENERATING NFTS FOR ALL UNIQUE ELEPHANTS")
    
    nft_results = []
    for i, elephant_unique_id in enumerate(elephant_id_system.elephants_db.keys(), 1):
        print(f"\n{'='*20} NFT {i}/{len(elephant_id_system.elephants_db)} {'='*20}")
        
        # Find face image for this elephant
        elephant_image = None
        for img_data in elephant_id_system.all_processed_images:
            if img_data['unique_id'] == elephant_unique_id:
                # Find corresponding result with aligned face
                for result in results:
                    if (result['image_path'] == img_data['image_path'] and 
                        'aligned_face' in result):
                        elephant_image = result['aligned_face']
                        print_result("Face Image Found", "Yes")
                        break
                if elephant_image is not None:
                    break
        
        if elephant_image is None:
            print_result("Face Image Found", "No - Using artistic silhouette only")
        
        # Generate NFT
        nft_info = nft_generator.generate_nft_from_elephant_id(
            elephant_unique_id,
            face_image=elephant_image,
            upload_to_ipfs=True  # Simulate IPFS upload
        )
        
        nft_results.append(nft_info)
        
        print_result(f"NFT for {elephant_unique_id}", "COMPLETED")
    
    # Final comprehensive summary
    print_section_header("FINAL SYSTEM SUMMARY")
    
    print_result("Images Processed", len(results))
    print_result("Unique Elephants Identified", len(elephant_id_system.elephants_db))
    print_result("NFTs Generated", len(nft_results))
    print_result("Total Embeddings in Database", sum(len(embs) for embs in elephant_id_system.elephants_db.values()))
    
    # Show thesis comparison directories
    print("\n" + "-" * 80)
    print("THESIS COMPARISON IMAGES")
    print("-" * 80)
    
    alignment_files = list(elephant_id_system.alignment_comparison_dir.glob("*.jpg"))
    augmentation_files = list(elephant_id_system.augmentation_comparison_dir.glob("*.jpg"))
    
    print_result("Face Alignment Comparisons", f"{len(alignment_files)} images saved")
    print_result("View Augmentation Comparisons", f"{len(augmentation_files)} images saved")
    print_result("Alignment Comparison Directory", elephant_id_system.alignment_comparison_dir)
    print_result("Augmentation Comparison Directory", elephant_id_system.augmentation_comparison_dir)
    
    # Show sample files for reference
    if alignment_files:
        print("\n  Sample Alignment Comparison Files:")
        for i, file_path in enumerate(alignment_files[:5]):  # Show first 5
            print(f"    {i+1}. {file_path.name}")
        if len(alignment_files) > 5:
            print(f"    ... and {len(alignment_files) - 5} more files")
    
    if augmentation_files:
        print("\n  Sample Augmentation Comparison Files:")
        for i, file_path in enumerate(augmentation_files[:5]):  # Show first 5
            print(f"    {i+1}. {file_path.name}")
        if len(augmentation_files) > 5:
            print(f"    ... and {len(augmentation_files) - 5} more files")
    
    # Show all elephant IDs
    print("\n" + "-" * 80)
    print("ALL ELEPHANT IDs")
    print("-" * 80)
    
    for i, elephant_id in enumerate(elephant_id_system.elephants_db.keys(), 1):
        embedding_count = len(elephant_id_system.elephants_db[elephant_id])
        nft_info = next((nft for nft in nft_results if nft['elephant_id'] == elephant_id), None)
        nft_status = "✓ NFT Created" if nft_info else "✗ No NFT"
        
        print(f"{i:2d}. {elephant_id} | {embedding_count} embeddings | {nft_status}")
    
    # Output directories
    print("\n" + "-" * 80)
    print("OUTPUT DIRECTORIES")
    print("-" * 80)
    
    print_result("Elephant Models", elephant_id_system.model_dir)
    print_result("NFT Output", nft_generator.nft_output_dir)
    print_result("NFT Images", nft_generator.images_dir)
    print_result("NFT Metadata", nft_generator.metadata_dir)
    
    print_section_header("PROCESSING COMPLETE!")
    
    return results, nft_results

print(" Main execution function defined!")

# Cell 16: Run the System
if __name__ == "__main__":
    # Run the complete system
    results, nft_results = main()
    
    print(f"\n System execution completed!")
    print(f" Results summary:")
    print(f"   - Images processed: {len(results) if results else 0}")
    print(f"   - NFTs generated: {len(nft_results) if nft_results else 0}")

All libraries imported successfully!
Utility functions defined!
ElephantFaceIDDetailed class initialization defined!
Model and database loading methods defined!
Image preprocessing methods defined!
Face alignment with thesis comparison methods defined!
Augmentation comparison grid method defined!
View augmentation with thesis comparison methods defined!
Feature extraction methods defined!
Embedding and ID generation methods defined!
Matching and database methods defined!
Image processing and dataset methods defined!
NFT Generator class defined!
 Main execution function defined!

 ELEPHANT FACE ID & NFT SYSTEM - COMPLETE PROCESSING 

 INITIALIZING ELEPHANT FACE ID SYSTEM 
  ✓ Model Directory: elephant_models
  ✓ Dataset Path: D:\Downloads\Elephants
  ✓ Thesis Output Directory: thesis_comparisons
  ✓ Alignment Comparison Directory: thesis_comparisons\face_alignment_normalization
  ✓ Augmentation Comparison Directory: thesis_comparisons\view_augmentation_pose_invariance
  ✓ PCA Model Path